In [23]:
!python -m tdmclient list
!python -m pip install tk-tools
!python -m tdmclient gui

id:       f3394cac-4181-424e-86bd-f1910fa2563b
group id: 44d28aad-03e2-499b-9d65-dd1e35adcaaf
name:     Thymio 210
status:   2 (available)
cap:      7
firmware: 13



In [24]:
# Import tdmclient Notebook environment:
import tdmclient.notebook
await tdmclient.notebook.start()

import time
import math

In [25]:
import numpy as np
from numpy import zeros

In [110]:
# parametres
time_rotation = 7.4
speed_r = 125
speed_f = 125
stop = 0

periode = 4

path = [[150, 50], [150, 100], [100,150]]

In [90]:
@tdmclient.notebook.sync_var
def motors(l_speed=500, r_speed=500):
    """
    Sets the motor speeds of the Thymio 
    param l_speed: left motor speed
    param r_speed: right motor speed
    """
    global motor_left_target, motor_right_target
    
    motor_left_target = l_speed
    motor_right_target = r_speed

In [91]:
#input : angle to change the trajectory
@tdmclient.notebook.sync_var
def orientation (angle):
    
    global motor_left_target, motor_right_target
    
    if angle < 0:
        motors(speed_r, stop)
    else :
        motors(stop, speed_r)
        
    time.sleep(time_rotation * abs(angle) / (math.pi))
    motors(stop, stop)

In [122]:
#input : path, table of the points to reach 
#return : teta, table of the angles taken by the robot to reach each point
def diff_angle (path):
    i = 1
    teta = zeros(np.shape(path)[0])
    
    teta[0] = math.atan2(path[0][0], path[0][1])
    
    while i < (np.shape(path)[0]):
        if path[i][1] != path[i-1][1] :
            teta [i] = math.atan2((path[i][0] - path[i-1][0]), (path[i][1] - path[i-1][1]))
        else : 
            teta[i] = teta[i-1]
        i = i + 1
    return teta

In [93]:
#input : coordinates of two different points 
#return the distance between two points 
def distance (x1, y1, x2, y2):
    dist = math.sqrt((x1-x2)*(x1-x2) + (y1-y2)*(y1-y2))
    return dist

In [123]:
def mouving (path):
    teta = zeros(np.shape(path)[0])
    teta = diff_angle(path)
    goalx = path[0][0]
    goaly = path[0][1]
    
    i = 0
    x_pense = 0
    y_pense = 0
    teta_pense = 0
    
    deltax = 0
    deltay = 0
    deltateta = 0
    
    orientation(teta[0]-teta_pense) #teta at t=0 given by kalman

    while i < np.shape(path)[0]-1:
        #[deltax, delaty, deltateta] = kalman()
        teta[i] = math.atan2((goalx - (x_pense + deltax)), (goaly - (y_pense + deltay)))
        dist = distance (goalx, goaly, x_pense + deltax, y_pense + deltay)
        if (dist < (speed_f * periode)):
            i = i + 1
            goalx = path[i][0]
            goaly = path[i][1]
            motors(speed_f, speed_f)
            time.sleep(dist/(speed_f/3))
            orientation(teta[i] - teta[i-1])
        else :
            orientation(teta[i] - (teta_pense + deltateta))
            motors(speed_f, speed_f)
            x_pense = x_pense + deltax + speed_f * periode * math.cos(teta[i])
            y_pense = y_pense + deltay + speed_f * periode * math.sin(teta[i])
            teta_pense = teta[i]
            time.sleep(periode)
   
    
    # move the robot to the last point
    while x_pense!=path[i][0] and y_pense!=path[i][1]:
        dist = distance(path[i][0], path[i][1], x_pense + deltax, y_pense + deltay)
        if (dist < speed_f * periode):
            motors(speed_f, speed_f)
            time.sleep(dist/(speed_f/3))
            x_pense=path[i][0]
            y_pense=path[i][1]
        else :
            motors(speed_f, speed_f)
            x_pense = x_pense + deltax + speed_f * periode * math.cos(teta[i])
            y_pense = y_pense + deltay + speed_f * periode * math.sin(teta[i])
            time.sleep(periode)
            
    motors(stop, stop)

In [124]:
mouving(path)

[[150, 50], [150, 100], [100, 150]]
[ 1.24904577  0.         -0.78539816]


In [121]:
motors(stop, stop)